In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_excel('AB_NYC_2019.xlsx')

# Display first few rows
print('First 5 rows of the dataset:')
print(df.head())

# Check dataset info
print('\nDataset Info:')
print(df.info())

# Check for missing values
print('\nMissing Values:')
print(df.isnull().sum())

# Check for duplicates
print('\nNumber of Duplicate Rows:')
print(df.duplicated().sum())

ModuleNotFoundError: No module named 'matplotlib.backends.registry'

## Step 2: Handle Missing, Duplicate, and Inconsistent Values

We'll address:
- **Missing Values**: Columns like `name`, `host_name`, `last_review`, and `reviews_per_month` may have missing values. We'll decide whether to fill or drop them based on relevance.
- **Duplicates**: Remove any duplicate rows if present.
- **Inconsistent Values**: Check for outliers or invalid entries (e.g., `price` <= 0, negative `minimum_nights`).

In [ ]:
# Handle missing values
# Drop 'name' and 'host_name' as they are not critical for analysis
df = df.drop(['name', 'host_name'], axis=1)

# Fill missing 'last_review' with a placeholder date (e.g., dataset start)
df['last_review'] = df['last_review'].fillna('1900-01-01')

# Fill missing 'reviews_per_month' with 0 (no reviews)
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

# Verify missing values are handled
print('Missing Values After Handling:')
print(df.isnull().sum())

# Remove duplicates if any
df = df.drop_duplicates()
print('\nNumber of Duplicate Rows After Removal:')
print(df.duplicated().sum())

# Check for inconsistent values
# Remove rows with price <= 0
df = df[df['price'] > 0]

# Remove rows with negative or unrealistic minimum_nights (e.g., > 365)
df = df[(df['minimum_nights'] > 0) & (df['minimum_nights'] <= 365)]

# Check for outliers in price using IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['price'] >= Q1 - 1.5 * IQR) & (df['price'] <= Q3 + 1.5 * IQR)]

print('\nDataset Shape After Cleaning:')
print(df.shape)

## Step 3: Convert Categorical Variables

We'll encode categorical variables (`neighbourhood_group`, `neighbourhood`, `room_type`) using LabelEncoder for simplicity. For high-cardinality columns like `neighbourhood`, we could use target encoding in a modeling context, but LabelEncoder suffices for preprocessing.

In [ ]:
# Encode categorical variables
le = LabelEncoder()

# Encode 'neighbourhood_group'
df['neighbourhood_group'] = le.fit_transform(df['neighbourhood_group'])
print('\nUnique neighbourhood_group values:', df['neighbourhood_group'].unique())

# Encode 'neighbourhood'
df['neighbourhood'] = le.fit_transform(df['neighbourhood'])
print('Unique neighbourhood values:', df['neighbourhood'].unique())

# Encode 'room_type'
df['room_type'] = le.fit_transform(df['room_type'])
print('Unique room_type values:', df['room_type'].unique())

# Verify encoding
print('\nDataset Head After Encoding:')
print(df.head())

## Step 4: Handle Skewed Distributions

The `price` column is often right-skewed in Airbnb datasets. We'll apply a log-transformation to normalize it and visualize the distribution before and after.

In [ ]:
# Visualize original price distribution
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(df['price'], kde=True, color='blue')
plt.title('Price Distribution (Original)')
plt.xlabel('Price')
plt.ylabel('Frequency')

# Apply log-transformation to price
df['price_log'] = np.log1p(df['price'])  # log1p handles zero values

# Visualize log-transformed price distribution
plt.subplot(1, 2, 2)
sns.histplot(df['price_log'], kde=True, color='green')
plt.title('Price Distribution (Log-Transformed)')
plt.xlabel('Log(Price)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

# Drop original price column
df = df.drop('price', axis=1)

## Step 5: Drop Irrelevant or Highly Correlated Features

We'll:
- **Drop Irrelevant Features**: Columns like `id`, `host_id`, and `last_review` are not useful for most analyses.
- **Check for High Correlation**: Use a correlation matrix to identify and drop highly correlated features (e.g., correlation > 0.8).

In [ ]:
# Drop irrelevant columns
df = df.drop(['id', 'host_id', 'last_review'], axis=1)

# Calculate correlation matrix
corr_matrix = df.corr()

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

# Identify highly correlated features (> 0.8)
high_corr = [(col1, col2) for col1 in corr_matrix for col2 in corr_matrix if (corr_matrix.loc[col1, col2] > 0.8) and (col1 != col2)]
print('\nHighly Correlated Features (> 0.8):')
print(high_corr if high_corr else 'None')

# No highly correlated features in this dataset, so no additional drops needed

# Final dataset info
print('\nFinal Dataset Info:')
print(df.info())
print('\nFinal Dataset Head:')
print(df.head())

## Conclusion

- **Missing Values**: Dropped `name` and `host_name` as non-essential; filled `last_review` and `reviews_per_month` appropriately.
- **Duplicates and Inconsistencies**: Removed duplicates and invalid entries (e.g., `price` <= 0, unrealistic `minimum_nights`).
- **Categorical Variables**: Encoded `neighbourhood_group`, `neighbourhood`, and `room_type` using LabelEncoder.
- **Skewed Distributions**: Log-transformed `price` to normalize its distribution.
- **Feature Selection**: Dropped irrelevant columns (`id`, `host_id`, `last_review`) and confirmed no highly correlated features.

The preprocessed dataset is now clean and ready for further analysis or modeling.